In [4]:
import pandas as pd
import json

# ====================================================
# 1. CẤU HÌNH DANH SÁCH TOP ITEMS (Dùng cho Cold Start)
# ====================================================
# Đây là danh sách 10 sản phẩm bán chạy nhất chúng ta tìm được ở bước trước
# (Bạn có thể thay thế list này bằng list kết quả từ code tìm Top 10 của bạn nếu khác)
TOP_ITEMS_COLD_START = ['4690000000001', '2803000000013', '5950000000001', '2803000000011', '0203000000004', '2803000000012', '1512000000004', '0020090000042', '2231000000030', '2231000000029']
#'4690000000001' - 
#'0203000000004'
#'5950000000001' - 1512000000004(mua cặp rất nhìu), số lượng:4236
#'2803000000013' - '2803000000011', '2803000000012' số lượng: rất lớn, có thể có 2803000000010: 4626
# '2231000000030', '2231000000029'
# trend tháng 1/2025: 280000138, sl: 1351
# ====================================================
# 2. ĐỌC DỮ LIỆU
# ====================================================
print("⏳ Đang đọc file Ground Truth...")
gt_df = pd.read_csv('final_groundtruth.csv')

# Tạo danh sách các User BẮT BUỘC phải trả kết quả
# Ép kiểu sang string để so khớp chính xác với JSON
required_users = set(gt_df['customer_id'].astype(str))
print(f"✅ Tìm thấy {len(required_users)} users cần dự đoán trong file CSV.")

input_json = "pred_feb2025.json"
output_json = "pred_submission (5).json"

try:
    with open(input_json, 'r') as f:
        pred_data = json.load(f) # pred_data là dictionary {userID: [item1, item2...]}
    
    print(f"📂 Đã load file JSON gốc. Tổng số user có dự đoán: {len(pred_data)}")
    
    # ====================================================
    # 3. XỬ LÝ: LỌC + ĐIỀN KHUYẾT (CORE LOGIC)
    # ====================================================
    final_submission = {}
    filled_count = 0
    
    # Duyệt qua từng user trong danh sách BẮT BUỘC (Ground Truth)
    for user in required_users:
        if user in pred_data:
            # TRƯỜNG HỢP 1: Đã có dự đoán từ model -> Giữ nguyên
            final_submission[user] = pred_data[user]
        else:
            # TRƯỜNG HỢP 2: User mới (Cold Start) -> Điền Top Items
            final_submission[user] = TOP_ITEMS_COLD_START
            filled_count += 1
            
    # ====================================================
    # 4. LƯU KẾT QUẢ
    # ====================================================
    print(f"✅ Đã xử lý xong!")
    print(f"   - Tổng user trong file kết quả: {len(final_submission)}")
    print(f"   - Số user dùng Model dự đoán: {len(final_submission) - filled_count}")
    print(f"   - Số user Cold-start (được điền Top Items): {filled_count}")
    
    with open(output_json, 'w') as f:
        json.dump(final_submission, f)
        
    print(f"💾 Đã lưu file hoàn chỉnh: {output_json}")
    
except FileNotFoundError:
    print(f"❌ Không tìm thấy file '{input_json}'. Hãy kiểm tra lại tên file.")

⏳ Đang đọc file Ground Truth...
✅ Tìm thấy 644970 users cần dự đoán trong file CSV.
📂 Đã load file JSON gốc. Tổng số user có dự đoán: 1324881
✅ Đã xử lý xong!
   - Tổng user trong file kết quả: 644970
   - Số user dùng Model dự đoán: 430675
   - Số user Cold-start (được điền Top Items): 214295
💾 Đã lưu file hoàn chỉnh: pred_submission (5).json
